In [18]:
#collect here all imports and installs
import cv2
import os
import subprocess
import numpy as np
import pandas as pd
import time



In [19]:
#without motion score
#Table created in: 188.67073893547058 seconds
    
#with spatial and temporal info
#Table created in: 629.0842838287354 seconds

#with motion score
#Table created in: 3010.119035959244 seconds


In [20]:
#Run to initiate functions 


def CreateFeatureTable(directory):
    #Create Pandas table 
    
    # Start the timer
    start_time = time.time()
    # Create the pandas DataFrame with column name is provided explicitly
    df = pd.DataFrame()
    
    filenames = []
    bitrates = [] 
    width = []
    height = []
    fps = [] #frames Per Second, should we round?
    
    average_intensity = []

    average_brightness = []
    average_contrast = [] 
    average_sharpness = []
    motion_score = []
    spatial_info = []
    temporal_info = []


    # iterate over files in
    # that directory
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f):
            filenames.append(f.replace('videos/', ''))
            b = GetBitrate(f)
            bitrates.append(b)
            average_intensity.append(AverageSpatialValue(f))
            SP = AverageSpatialInformation(f)
            average_brightness.append(SP[0])
            average_contrast.append(SP[1])
            average_sharpness.append(SP[2])
            #motion_score.append(calculateMotionScore(f))
            
            properties = get_video_properties(f)
            width.append(properties[0])
            height.append(properties[1])
            fps.append(properties[2])
            spatial_info.append(calculate_spatial_information(f))
            temporal_info.append(calculate_temporal_information(f))
            


      

    df = df.assign(Name=filenames,
                   Bitrate=bitrates,
                   width=width,
                   height=height,
                   spatial_info=spatial_info,
                   temporal_info=temporal_info,
                   average_intensity=average_intensity,
                   average_brightness = average_brightness,
                   average_contrast = average_contrast,
                   average_sharpness = average_sharpness,
                   #motion_score = motion_score,
                   
                  # fps=fps why is not working??
                  )
    # End the timer
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time

    print(f"Table created in: {elapsed_time} seconds")
  
    return df
    
#Find video properties 


def get_video_properties(video_path):
    # Open the video file
    video = cv2.VideoCapture(video_path)

    # Get the video width and height
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Get the frames per second (FPS)
    fps = video.get(cv2.CAP_PROP_FPS)

    # Release the video file
    video.release()

    return width, height, fps


#Calculate the bitrate 

def GetBitrate(video_path):
    
    # Start the timer
    #start_time = time.time()
    
    # Get video file size in bytes
    file_size = os.path.getsize(video_path)

    # Get video duration in seconds using FFprobe
    ffprobe_cmd = f'ffprobe -i "{video_path}" -show_entries format=duration -v quiet -of csv="p=0"'
    result = subprocess.run(ffprobe_cmd, capture_output=True, text=True, shell=True)
    duration = float(result.stdout.strip())

    # Calculate the estimated bitrate in kilobits per second (kbps)
    bitrate = (file_size * 8) / (duration * 1000)
    
    # End the timer
    #end_time = time.time()

    # Calculate the elapsed time
    #elapsed_time = end_time - start_time

    #print(f"Bitrate calculated in: {elapsed_time} seconds")
    
    return bitrate

    #print(f"Video Bitrate: {bitrate:.2f} kbps")

    
    

def calculate_spatial_information(video_path):
    video = cv2.VideoCapture(video_path)
    sobel_stds = []

    while True:
        ret, frame = video.read()

        if not ret:
            break

        # Convert the frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Apply the Sobel filter to the luminance component
        sobel_x = cv2.Sobel(gray_frame, cv2.CV_64F, 1, 0, ksize=3)
        sobel_y = cv2.Sobel(gray_frame, cv2.CV_64F, 0, 1, ksize=3)
        sobel = np.sqrt(sobel_x**2 + sobel_y**2)

        # Calculate the standard deviation of the Sobel filter
        sobel_std = np.std(sobel)

        sobel_stds.append(sobel_std)

    video.release()

    # Calculate the maximum standard deviation
    max_std = max(sobel_stds)

    return max_std

def calculate_temporal_information(video_path):
    video = cv2.VideoCapture(video_path)
    motion_diffs = []

    ret, prev_frame = video.read()

    while True:
        ret, curr_frame = video.read()

        if not ret:
            break

        # Convert frames to grayscale
        prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
        curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)

        # Calculate the motion difference between frames
        motion_diff = curr_gray - prev_gray

        # Calculate the standard deviation of the motion difference
        motion_std = np.std(motion_diff)

        motion_diffs.append(motion_std)

        prev_frame = curr_frame

    video.release()
    # Calculate the maximum standard deviation
    max_std = max(motion_diffs)

    return max_std


#WHAT IT GIVES US?

#Average Spatial Value

def AverageSpatialValue(video_path):
    # Start the timer
    #start_time = time.time()
    
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if the video file is successfully opened
    if not cap.isOpened():
        print("Error opening video file")
        exit()

    # Initialize variables
    frame_count = 0
    total_intensity = 0

    # Read and process frames
    while True:
        # Read a frame from the video
        ret, frame = cap.read()

        # If the frame was not read successfully, then we have reached the end of the video
        if not ret:
            break

        # Calculate the average pixel intensity of the frame
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        intensity = np.mean(gray_frame)

        # Accumulate the total intensity
        total_intensity += intensity

        # Display the frame with intensity value (optional)
        #cv2.putText(frame, f"Intensity: {intensity:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        #cv2.imshow('Frame with Intensity', frame)
        #if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit
        #    break

        # Increment frame count
        frame_count += 1

    # Calculate the average spatial value
    average_intensity = total_intensity / frame_count

    # Release the video capture object and close any open windows
    cap.release()
    cv2.destroyAllWindows()
    
    # End the timer
    #end_time = time.time()

    # Calculate the elapsed time
    #elapsed_time = end_time - start_time

    #print(f"Average Spatial Value calculated in: {elapsed_time} seconds")
    
    return average_intensity
    #print(f"Average Spatial Value: {average_intensity}")
    
    
    
#Calculate average spatial information


import cv2
def AverageSpatialInformation(video_path):

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Initialize variables to store spatial information
    frame_count = 0
    total_brightness = 0.0
    total_contrast = 0.0
    total_sharpness = 0.0

    # Iterate over the frames
    while True:
        # Read the next frame
        ret, frame = cap.read()

        # If no more frames, break the loop
        if not ret:
            break

        # Convert the frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Calculate brightness (average pixel intensity)
        brightness = gray_frame.mean()
        total_brightness += brightness
        #print('calculate brightness')
        # Calculate contrast (standard deviation of pixel intensities)
        contrast = gray_frame.std()
        total_contrast += contrast
        #print('calculate contrast')
        # Calculate sharpness (Laplacian variance)
        laplacian_var = cv2.Laplacian(gray_frame, cv2.CV_64F).var()
        total_sharpness += laplacian_var
        #print('calculate sharpness')

        # Increment frame count
        frame_count += 1

    # Release the video capture
    cap.release()

    # Calculate average spatial information
    average_brightness = total_brightness / frame_count
    average_contrast = total_contrast / frame_count
    average_sharpness = total_sharpness / frame_count
    
    return average_brightness, average_contrast, average_sharpness

    #print(frame_count)
    # Print the spatial information measures
    print(f"Average Brightness: {average_brightness}")
    print(f"Average Contrast: {average_contrast}")
    print(f"Average Sharpness: {average_sharpness}")



In [21]:
#TAKES TOO LONG

#Analyse motion
#Returns video Motion Score


def calculateMotionScore(video_path):

    # Start the timer
    #start_time = time.time()

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Read the first frame
    ret, prev_frame = cap.read()

    # Convert the frame to grayscale
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

    # Initialize motion vectors
    motion_vectors = []

    # Set parameters for Farneback optical flow
    params = dict(pyr_scale=0.5, levels=3, winsize=15, iterations=3, poly_n=5, poly_sigma=1.2, flags=0)

    # Iterate over the frames
    while True:
        # Read the next frame
        ret, frame = cap.read()

        # If no more frames, break the loop
        if not ret:
            break

        # Convert the frame to grayscale
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Calculate optical flow using Farneback algorithm
        flow = cv2.calcOpticalFlowFarneback(prev_frame_gray, frame_gray, None, **params)

        # Calculate the magnitude of motion vectors
        magnitude = np.sqrt(flow[..., 0] ** 2 + flow[..., 1] ** 2)

        # Calculate the average magnitude
        average_magnitude = np.mean(magnitude)

        # Store the average magnitude as a motion value
        motion_vectors.append(average_magnitude)

        # Set the current frame as the previous frame for the next iteration
        prev_frame_gray = frame_gray.copy()

    # Release the video capture
    cap.release()

    # Calculate the overall motion score
    motion_score = np.mean(motion_vectors)



    # End the timer
    #end_time = time.time()

    # Calculate the elapsed time
    #elapsed_time = end_time - start_time

    #print(f"Elapsed Time too find motion score: {elapsed_time} seconds")

    #print(f"Motion Score: {motion_score}")
    return motion_score


In [22]:
# assign directory
directory = 'videos'

# Start the timer
start_time = time.time()
feature_matrix = CreateFeatureTable(directory);
print(feature_matrix);
feature_matrix.to_csv ('feature_output/features.csv', index = False, header=True)

# End the timer
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print(f"Table created in: {elapsed_time} seconds")

Table created in: 225.40843391418457 seconds
                                   Name       Bitrate  width  height  \
0   big_buck_bunny_720p24-Scene-002.mp4  78676.570839   1280     720   
1   big_buck_bunny_720p24-Scene-016.mp4  35535.435877   1280     720   
2   big_buck_bunny_720p24-Scene-017.mp4  38315.829029   1280     720   
3   big_buck_bunny_720p24-Scene-003.mp4  36179.684015   1280     720   
4   big_buck_bunny_720p24-Scene-015.mp4  33525.657021   1280     720   
5   big_buck_bunny_720p24-Scene-001.mp4  51249.820968   1280     720   
6   big_buck_bunny_720p24-Scene-029.mp4  35771.808042   1280     720   
7   big_buck_bunny_720p24-Scene-028.mp4  43341.883047   1280     720   
8   big_buck_bunny_720p24-Scene-014.mp4  40821.179036   1280     720   
9   big_buck_bunny_720p24-Scene-038.mp4  44505.685598   1280     720   
10  big_buck_bunny_720p24-Scene-010.mp4  34074.829219   1280     720   
11  big_buck_bunny_720p24-Scene-004.mp4  72856.044313   1280     720   
12  big_buck_bunny_

In [25]:
def calc_Bitrate(directory):
    bitrates = []
    filenames = []
    df = pd.DataFrame()
    for filename in os.listdir(directory):
            f = os.path.join(directory, filename)
            # checking if it is a file
            if os.path.isfile(f):
                  filenames.append(f.replace('videos_sd/', ''))
                  bitrates.append(GetBitrate(f))
    df = df.assign(Name=filenames, Bitrate=bitrates)
    return df



bitrates=calc_Bitrate('videos_sd')
print(bitrates)
bitrates.to_csv('bitrates.csv')

                                   Name      Bitrate
0   big buck bunny 720p24 Scene 049.mp4  2001.957490
1   big buck bunny 720p24 Scene 048.mp4   556.689637
2   big buck bunny 720p24 Scene 007.mp4  1057.504177
3   big buck bunny 720p24 Scene 013.mp4  1071.508501
4   big buck bunny 720p24 Scene 012.mp4  2299.811207
5   big buck bunny 720p24 Scene 006.mp4   481.478944
6   big buck bunny 720p24 Scene 010.mp4  1055.434705
7   big buck bunny 720p24 Scene 004.mp4  2039.040609
8   big buck bunny 720p24 Scene 038.mp4  1752.416025
9   big buck bunny 720p24 Scene 039.mp4  3009.050443
10  big buck bunny 720p24 Scene 005.mp4  1891.498429
11  big buck bunny 720p24 Scene 011.mp4  1953.028908
12  big buck bunny 720p24 Scene 029.mp4   739.814792
13  big buck bunny 720p24 Scene 015.mp4   725.144206
14  big buck bunny 720p24 Scene 001.mp4  1445.208976
15  big buck bunny 720p24 Scene 014.mp4  1525.316164
16  big buck bunny 720p24 Scene 028.mp4  1653.285318
17  big buck bunny 720p24 Scene 002.mp4  2275.